# A) Scrapper Top 50 des personnalités politiques français sur Twitter

In [276]:
!pip install beautifulsoup4
import requests
from bs4 import BeautifulSoup

In [277]:
# Déclarer de la liste des comptes
compte_twitter = []
stopwords = []

# Faire la demande "GET" à une URL
r = requests.get('http://ymobactus.miaouw.net/labo-top-politiques.php?mode=followers&liste=personnalites&tendance')

# Extraire le contenu
d = r.content

# Créer un objet soupe
soup = BeautifulSoup(d)

# Récupérer du "href" contenant le lien url redirigeant vers le compte Twitter de chaque personnalité
handles = [ a["href"] for a in soup.find_all("a", href=True) if("twitter" in a["href"])]

# Nous bouclons afin de ne récupérer que la fin de url (nom du compte)
#Pour d'avoir uniquement le nom du compte, nous utilison REGEX 

for url in handles: 
    compte = re.search('[^/]+(?=/$|$)',url) 
# Ajouter des noms à la liste compte_twitter 
    compte_twitter.append(compte.group())

In [278]:
print(compte_twitter)

['lists', 'topolitiq', 'EmmanuelMacron', 'MLP_officiel', 'NicolasSarkozy', 'fhollande', 'JLMelenchon', 'Anne_Hidalgo', 'manuelvalls', 'alainjuppe', 'najatvb', 'EPhilippe_LH', 'benoithamon', 'ChTaubira', 'nk_m', 'RoyalSegolene', 'Lagarde', 'bayrou', 'fleurpellerin', 'FrancoisFillon', 'BrunoLeMaire', 'montebourg', 'CecileDuflot', 'MarionMarechal', 'vpecresse', 'CCastaner', 'laurentwauquiez', 'jf_cope', 'olivierveran', 'JeanCASTEX', 'jeanmarcayrault', 'LaurentFabius', 'GDarmanin', 'BGriveaux', 'GilbertCollard', 'dupontaignan', 'f_philippot', 'nadine__morano', 'MichelBarnier', 'BCazeneuve', 'pierremoscovici', 'JLBorloo', 'xavierbertrand', 'cestrosi', 'edgarmorinparis', 'axellelemaire', 'ECiotti', 'aurelifil', 'jpraffarin', 'lepenjm', 'MartineAubry', 'datirachida', 'ymobactus', 'ymobactus']


# B) Scrapper des tweets sur Twitter

In [279]:
!pip install twint
!pip install --user --upgrade git+https://github.com/twintproject/twint.git@origin/master#egg=twint
!pip install Owlready2
!pip install treetaggerwrapper

  Cloning https://github.com/twintproject/twint.git (to revision origin/master) to c:\users\phamd\appdata\local\temp\pip-install-tqo2ox5b\twint_830ccafbf3864d2aa5d3f28ac193c8ce


  Running command git clone -q https://github.com/twintproject/twint.git 'C:\Users\phamd\AppData\Local\Temp\pip-install-tqo2ox5b\twint_830ccafbf3864d2aa5d3f28ac193c8ce'
  ERROR: Error [WinError 2] The system cannot find the file specified while executing command git clone -q https://github.com/twintproject/twint.git 'C:\Users\phamd\AppData\Local\Temp\pip-install-tqo2ox5b\twint_830ccafbf3864d2aa5d3f28ac193c8ce'
ERROR: Cannot find command 'git' - do you have 'git' installed and in your PATH?


In [280]:
import twint
import requests
import pandas as pd
import nest_asyncio
import numpy as np

In [281]:
# Configure
c = twint.Config()
c.Username = "JLMelenchon"
c.Limit = 10
c.Store_csv = True
c.Output = "tweets.csv"
# Courir
nest_asyncio.apply()
twint.run.Search(c)

1401629165086982145 2021-06-06 21:56:48 +0200 <JLMelenchon> L’union populaire : une stratégie globale  https://t.co/Fuadnr7wdF
1401628337500475393 2021-06-06 21:53:31 +0200 <JLMelenchon> D'un buzz affligeant. La post-vérité en action.   ➡️  https://t.co/ZxPJqD3iVr  https://t.co/McA399lRjm
1401628174883168261 2021-06-06 21:52:52 +0200 <JLMelenchon> Le Parisien donne la parole à un complotiste islamogauchiste ?    https://t.co/DYV9k8Yf68
1401625261876125705 2021-06-06 21:41:18 +0200 <JLMelenchon> France Inter complotiste ????    https://t.co/MQRcj5ydCu
1401602168969383938 2021-06-06 20:09:32 +0200 <JLMelenchon> L'horrible article complotiste du Figaro qui m'a inspiré. Publié juste après l'attaque sur les Champs-Élysées.   https://t.co/qS0tDKuAYy
1401598967847305217 2021-06-06 19:56:49 +0200 <JLMelenchon> Les complotistes anticomplotistes sont de sortie. Ils nient que les assassins font leur coup au moment qui fait parler d'eux. Propos ineptes. À moins que ce soit pour les couvrir.
140152

# C) Analyse de sentiment des tweets représentant les émotions détectées

In [282]:
import pandas as pd
import io
import requests
url="http://advanse.lirmm.fr/FEEL.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')),  sep = ';')

In [283]:
import re, string, unicodedata
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\phamd\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [284]:
def remove_URL(sample):
    """Remove URLs from a sample string"""
    return re.sub(r"https\S+", "", sample)

def remove_stopword(words):
    return [word for word in words.split() if word not in stopwords.words('french')]

def preprocess(txt):
    txt = txt.lower()
    txt = remove_URL(txt)
    txt = re.sub(r'\W', ' ', txt) # Remove all the special characters
    txt = "".join(filter(lambda x: not x.isdigit(), txt)) #remove number
    txt = re.sub('[!@#$]', '', txt)
    
    token_after_removing_stopword = remove_stopword(txt)
    
    txt = ' '.join(token_after_removing_stopword)

    return txt

In [285]:
df = pd.read_csv('tweets.csv', encoding='utf8')
feel = pd.read_csv('C:/Users/phamd/Downloads/FEEL.csv', encoding='utf8', delimiter=';')

df['tweet'].head()

0    L'horrible article complotiste du Figaro qui m...
1    Les complotistes anticomplotistes sont de sort...
2    #Véran efface 1936, 1947, 1968, 1981... Et Pét...
3    Biden est un point d'appui pour moi   #Questio...
4    Il y a eu une vaste opération de dédiabolisati...
Name: tweet, dtype: object

# Treetagger lemmatiseur

In [286]:
# TREETAGGER LEMMATIZER
import pandas as pd
import treetaggerwrapper as tt
  
t_tagger = tt.TreeTagger(TAGLANG ='fr', TAGDIR ='C:\TreeTagger')
df['clean_tweet'] = df['tweet'].apply(lambda txt: preprocess(txt))
df['pos_tags'] = df['clean_tweet'].apply(lambda txt: t_tagger.tag_text(txt))

In [287]:
df['pos_tags'].head()

0    [horrible\tADJ\thorrible, article\tNOM\tarticl...
1    [complotistes\tNOM\tcomplotistes, anticomploti...
2    [véran\tNOM\tvéran, efface\tVER:pres\teffacer,...
3    [biden\tNOM\tbiden, point\tADV\tpoint, appui\t...
4    [a\tVER:pres\tavoir, vaste\tADJ\tvaste, opérat...
Name: pos_tags, dtype: object

# Lemmatisation et POS

In [288]:
original = []
lemmas = []
tags = []

sentences_pos_tags = df['pos_tags'].tolist()

res = pd.DataFrame(columns=['original', 'lemma', 'tag'])

for pos_tags in sentences_pos_tags:
    original = []
    lemmas = []
    tags = []
    for t in pos_tags:
        original.append(t.split('\t')[0])
        tags.append(t.split('\t')[1])
        lemmas.append(t.split('\t')[-1])
    res = res.append({'original': original, 'lemma': lemmas, 'tag': tags}, ignore_index=True)
res

,original,lemma,tag
0,"[horrible, article, complotiste, figaro, a, in...","[horrible, article, complotiste, figaro, avoir...","[ADJ, NOM, ADJ, NOM, VER:pres, VER:pper, VER:p..."
1,"[complotistes, anticomplotistes, sortie, nient...","[complotistes, anticomplotistes, sortie, nier,...","[NOM, ADJ, NOM, VER:pres, ADJ, VER:pres, NOM, ..."
2,"[véran, efface, pétain, grand, militaire, maur...","[véran, effacer, pétain, grand, militaire, mau...","[NOM, VER:pres, NOM, ADJ, ADJ, NOM, VER:ppre, ..."
3,"[biden, point, appui, questionspol]","[biden, point, appui, questionspol]","[NOM, ADV, NOM, NOM]"
4,"[a, vaste, opération, dédiabolisation, mme, pe...","[avoir, vaste, opération, dédiabolisation, mad...","[VER:pres, ADJ, NOM, NOM, NOM, ADJ, NOM, NOM, ..."
...,...,...,...
195,"[clemence_guette, raveaudgilles, richesflores,...","[clemence_guette, raveaudgilles, richesflores,...","[NOM, NOM, ADJ, NOM, NOM, NOM, NOM, ADJ, NOM, ..."
196,"[clemence_guette, raveaudgilles, richesflores,...","[clemence_guette, raveaudgilles, richesflores,...","[NOM, NOM, ADJ, VER:pper, NOM, VER:pres, VER:i..."
197,"[clemence_guette, raveaudgilles, faut, revenir...","[clemence_guette, raveaudgilles, falloir, reve...","[NOM, NOM, VER:pres, VER:infi, NOM, VER:pres, ..."
198,"[clemence_guette, raveaudgilles, richesse, pay...","[clemence_guette, raveaudgilles, richesse, pay...","[NOM, NOM, NOM, NOM, VER:cond, NOM, ADJ, VER:p..."


# Lexique émotion

In [289]:
# Télécharger feel.csv
feel = pd.read_csv('C:/Users/phamd/Downloads/FEEL.csv', encoding='utf8', delimiter=';')
feel['polarity'] = feel['polarity'].replace('positive', 1)
feel['polarity'] = feel['polarity'].replace('negative', -1)
words = feel[feel.columns[1]].tolist()
emotions = feel[feel.columns[2:]].to_numpy()

dict_emotion = dict(zip(words, emotions))
dict_emotion

{'à ce endroit là': array([1, 0, 0, 0, 0, 0, 0], dtype=int64),
 'à le hâte': array([-1,  0,  1,  0,  0,  1,  0], dtype=int64),
 'à part': array([-1,  0,  0,  1,  0,  0,  0], dtype=int64),
 'à pic': array([-1,  0,  1,  0,  0,  0,  0], dtype=int64),
 'à rallonge': array([-1,  0,  0,  1,  0,  0,  0], dtype=int64),
 'abasourdir': array([-1,  0,  0,  0,  0,  1,  0], dtype=int64),
 'ablation': array([-1,  0,  1,  0,  0,  0,  1], dtype=int64),
 'abominable': array([-1,  0,  1,  0,  0,  0,  1], dtype=int64),
 'abrupt': array([-1,  0,  1,  0,  0,  0,  0], dtype=int64),
 'absent': array([-1,  0,  1,  1,  0,  0,  0], dtype=int64),
 'absorber': array([1, 0, 0, 0, 0, 1, 0], dtype=int64),
 'absurde': array([-1,  0,  0,  0,  1,  1,  1], dtype=int64),
 'acceptable': array([1, 0, 0, 0, 0, 0, 0], dtype=int64),
 'accidentel': array([-1,  0,  1,  1,  0,  1,  0], dtype=int64),
 'accusateur': array([-1,  0,  1,  0,  1,  0,  1], dtype=int64),
 'accuser': array([-1,  0,  1,  1,  1,  0,  1], dtype=int64),
 'ac

In [290]:
feel = pd.read_csv('C:/Users/phamd/Downloads/FEEL.csv', encoding='utf8', delimiter=';')
feel

,id,word,polarity,joy,fear,sadness,anger,surprise,disgust
0,1,à ce endroit là,positive,0,0,0,0,0,0
1,2,à le hâte,negative,0,1,0,0,1,0
2,3,à part,negative,0,0,1,0,0,0
3,4,à pic,negative,0,1,0,0,0,0
4,5,à rallonge,negative,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...
14122,14125,zozoter,negative,0,1,1,1,0,0
14123,14126,merci,positive,1,0,0,0,0,0
14124,14127,remercier,positive,1,0,0,0,0,0
14125,14128,remerciment,positive,1,0,0,0,0,0


In [291]:
from nltk import word_tokenize

lemmas = res['lemma'].apply(lambda x: ' '.join(x)).tolist() #get lemma
lemmas[0:10]

['horrible article complotiste figaro avoir inspirer publier juste après attaque champ élysée',
 'complotistes anticomplotistes sortie nier assassin faire coup moment faire parler propos inepte moins couvrir',
 'véran effacer pétain grand militaire maurras briller écrivain préciser macron dire ps',
 'biden point appui questionspol',
 'avoir vaste opération dédiabolisation madame pen banalisation idée matin soir médias si parler ouvrier bâtiment mourir travail gens dire pouvoir être avoir problème social france questionspol',
 'falloir convergence force|forces politique social pourquoi organiser marche liberté organisation rassembler plus manifestation déjà prévu questionspol',
 'falloir travailler améliorer technique policier sujet prendre justice seul insoumis proposer plan redéployer moyen policer autre manière questionspol',
 'oui parler thème sécurité sujet central élection sujet central aller faire face aggravation brutal crise écologique aller faire rapport catastropher social av

In [292]:
# boucle for dans chaque tweet
for lemma in lemmas:
    tokens = [w for w in word_tokenize(lemma)] #tokenize tweet
    print('twwet', lemma)
    print('Token', tokens)
    break

twwet horrible article complotiste figaro avoir inspirer publier juste après attaque champ élysée
Token ['horrible', 'article', 'complotiste', 'figaro', 'avoir', 'inspirer', 'publier', 'juste', 'après', 'attaque', 'champ', 'élysée']


In [293]:
for lemma in lemmas:
    tokens = [w for w in word_tokenize(lemma)] #tokenize tweet
    
    n_pos = 0
    n_neg = 0
    
    count = 0
    
    vector = np.zeros(n_emotion)
    tmp_sentiment = {}
    
    for token in tokens:
        if token in dict_emotion:
            # étiqueté de polarité pour tweets
            if dict_emotion[token][0] == 1:
                label='positive'
                n_pos += 1
            else:
                label = 'negative'
                n_neg += 1
            print("({0} | {1})".format(token, label), end=', ')
            
            # étiqueté d'émotion pour tweets
            vector = np.add(vector,dict_emotion[token][1:], out=vector) # somme des vecteurs d'émotion
            # print(dict_emotion[token][1:]) décommentez pour déboguer
            count += 1
        else:
            print(token, end=', ')
            

(horrible | negative), (article | positive), complotiste, figaro, avoir, (inspirer | positive), (publier | positive), (juste | positive), après, attaque, (champ | positive), élysée, complotistes, anticomplotistes, (sortie | positive), (nier | negative), (assassin | negative), (faire | positive), (coup | negative), (moment | positive), (faire | positive), (parler | positive), propos, (inepte | negative), (moins | negative), (couvrir | positive), véran, (effacer | negative), pétain, (grand | positive), (militaire | positive), maurras, (briller | positive), (écrivain | positive), préciser, macron, dire, ps, biden, point, (appui | positive), questionspol, avoir, (vaste | positive), (opération | negative), dédiabolisation, (madame | positive), pen, banalisation, (idée | positive), (matin | positive), (soir | positive), (médias | positive), si, (parler | positive), (ouvrier | positive), (bâtiment | positive), (mourir | negative), (travail | positive), (gens | positive), dire, (pouvoir | nega

In [294]:
from nltk import word_tokenize

lemmas = res['lemma'].apply(lambda x: ' '.join(x)).tolist() #get lemma

emotion = ['joy','fear', 'sadness', 'angry', 'surprise', 'disgust']
n_emotion = len(emotion)

polarity = []
sentiment = [] # stocker le niveau d'émotion

# boucle for dans chaque tweet
for lemma in lemmas:
    tokens = [w for w in word_tokenize(lemma)] #tokenize tweet
    
    n_pos = 0
    n_neg = 0
    
    count = 0
    
    vector = np.zeros(n_emotion)
    tmp_sentiment = {}
    
    for token in tokens:
        if token in dict_emotion:
            # étiqueté de polarité pour tweets
            if dict_emotion[token][0] == 1:
                label='positive'
                n_pos += 1
            else:
                label = 'negative'
                n_neg += 1
            print("({0} | {1})".format(token, label), end=', ')
            
            # étiqueté d'émotion pour tweets
            vector = np.add(vector,dict_emotion[token][1:], out=vector) # sum emotion vectors
            # print(dict_emotion[token][1:]) décommentez pour déboguer
            count += 1
        else:
            print(token, end=', ')
            
    # émotion sentiment
    if count != 0:
        vector /= count # normaliser par le moyen
    for emotion_label, emotion_value in zip(emotion, vector):
        tmp_sentiment.update({emotion_label : float(emotion_value)}) # {'joy': 0.0, 'fear': 0.0, 'sadness': 0.0, 'angry': 0.16, 'surprise': 0.16, 'disgust': 0.0}
    
    sentiment.append(tmp_sentiment)
    
    # polarity
    if n_pos > n_neg:
        polarity.append('positive')
    elif n_pos < n_neg:
        polarity.append('negative')
    else:
        polarity.append('neutral')
        
        
    # Imprimer la sortie pour chaque tweet
    print()
    print("\n->Emotion: ", list(tmp_sentiment.items()))
    print("\n->Polarity: ", polarity[-1])
    print()
    print('>'*100)
    print()

(horrible | negative), (article | positive), complotiste, figaro, avoir, (inspirer | positive), (publier | positive), (juste | positive), après, attaque, (champ | positive), élysée, 

->Emotion:  [('joy', 0.0), ('fear', 0.16666666666666666), ('sadness', 0.0), ('angry', 0.0), ('surprise', 0.16666666666666666), ('disgust', 0.16666666666666666)]

->Polarity:  positive

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

complotistes, anticomplotistes, (sortie | positive), (nier | negative), (assassin | negative), (faire | positive), (coup | negative), (moment | positive), (faire | positive), (parler | positive), propos, (inepte | negative), (moins | negative), (couvrir | positive), 

->Emotion:  [('joy', 0.0), ('fear', 0.2727272727272727), ('sadness', 0.09090909090909091), ('angry', 0.36363636363636365), ('surprise', 0.09090909090909091), ('disgust', 0.18181818181818182)]

->Polarity:  positive

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

In [295]:
res['polarity'] = polarity
res

,original,lemma,tag,polarity
0,"[horrible, article, complotiste, figaro, a, in...","[horrible, article, complotiste, figaro, avoir...","[ADJ, NOM, ADJ, NOM, VER:pres, VER:pper, VER:p...",positive
1,"[complotistes, anticomplotistes, sortie, nient...","[complotistes, anticomplotistes, sortie, nier,...","[NOM, ADJ, NOM, VER:pres, ADJ, VER:pres, NOM, ...",positive
2,"[véran, efface, pétain, grand, militaire, maur...","[véran, effacer, pétain, grand, militaire, mau...","[NOM, VER:pres, NOM, ADJ, ADJ, NOM, VER:ppre, ...",positive
3,"[biden, point, appui, questionspol]","[biden, point, appui, questionspol]","[NOM, ADV, NOM, NOM]",positive
4,"[a, vaste, opération, dédiabolisation, mme, pe...","[avoir, vaste, opération, dédiabolisation, mad...","[VER:pres, ADJ, NOM, NOM, NOM, ADJ, NOM, NOM, ...",positive
...,...,...,...,...
195,"[clemence_guette, raveaudgilles, richesflores,...","[clemence_guette, raveaudgilles, richesflores,...","[NOM, NOM, ADJ, NOM, NOM, NOM, NOM, ADJ, NOM, ...",positive
196,"[clemence_guette, raveaudgilles, richesflores,...","[clemence_guette, raveaudgilles, richesflores,...","[NOM, NOM, ADJ, VER:pper, NOM, VER:pres, VER:i...",positive
197,"[clemence_guette, raveaudgilles, faut, revenir...","[clemence_guette, raveaudgilles, falloir, reve...","[NOM, NOM, VER:pres, VER:infi, NOM, VER:pres, ...",positive
198,"[clemence_guette, raveaudgilles, richesse, pay...","[clemence_guette, raveaudgilles, richesse, pay...","[NOM, NOM, NOM, NOM, VER:cond, NOM, ADJ, VER:p...",positive


In [296]:
emotion_df = pd.DataFrame.from_dict(sentiment) #create a dataframe 
emotion_df

,joy,fear,sadness,angry,surprise,disgust
0,0.000000,0.166667,0.000000,0.000000,0.166667,0.166667
1,0.000000,0.272727,0.090909,0.363636,0.090909,0.181818
2,0.000000,0.400000,0.200000,0.400000,0.000000,0.200000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.125000,0.125000,0.062500,0.125000,0.125000
...,...,...,...,...,...,...
195,0.000000,0.111111,0.000000,0.111111,0.111111,0.111111
196,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000
197,0.076923,0.000000,0.076923,0.000000,0.000000,0.076923
198,0.071429,0.214286,0.071429,0.142857,0.142857,0.071429


In [297]:
tweets = res.join(emotion_df) # join 2 dataframe
tweets

,original,lemma,tag,polarity,joy,fear,sadness,angry,surprise,disgust
0,"[horrible, article, complotiste, figaro, a, in...","[horrible, article, complotiste, figaro, avoir...","[ADJ, NOM, ADJ, NOM, VER:pres, VER:pper, VER:p...",positive,0.000000,0.166667,0.000000,0.000000,0.166667,0.166667
1,"[complotistes, anticomplotistes, sortie, nient...","[complotistes, anticomplotistes, sortie, nier,...","[NOM, ADJ, NOM, VER:pres, ADJ, VER:pres, NOM, ...",positive,0.000000,0.272727,0.090909,0.363636,0.090909,0.181818
2,"[véran, efface, pétain, grand, militaire, maur...","[véran, effacer, pétain, grand, militaire, mau...","[NOM, VER:pres, NOM, ADJ, ADJ, NOM, VER:ppre, ...",positive,0.000000,0.400000,0.200000,0.400000,0.000000,0.200000
3,"[biden, point, appui, questionspol]","[biden, point, appui, questionspol]","[NOM, ADV, NOM, NOM]",positive,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,"[a, vaste, opération, dédiabolisation, mme, pe...","[avoir, vaste, opération, dédiabolisation, mad...","[VER:pres, ADJ, NOM, NOM, NOM, ADJ, NOM, NOM, ...",positive,0.000000,0.125000,0.125000,0.062500,0.125000,0.125000
...,...,...,...,...,...,...,...,...,...,...
195,"[clemence_guette, raveaudgilles, richesflores,...","[clemence_guette, raveaudgilles, richesflores,...","[NOM, NOM, ADJ, NOM, NOM, NOM, NOM, ADJ, NOM, ...",positive,0.000000,0.111111,0.000000,0.111111,0.111111,0.111111
196,"[clemence_guette, raveaudgilles, richesflores,...","[clemence_guette, raveaudgilles, richesflores,...","[NOM, NOM, ADJ, VER:pper, NOM, VER:pres, VER:i...",positive,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000
197,"[clemence_guette, raveaudgilles, faut, revenir...","[clemence_guette, raveaudgilles, falloir, reve...","[NOM, NOM, VER:pres, VER:infi, NOM, VER:pres, ...",positive,0.076923,0.000000,0.076923,0.000000,0.000000,0.076923
198,"[clemence_guette, raveaudgilles, richesse, pay...","[clemence_guette, raveaudgilles, richesse, pay...","[NOM, NOM, NOM, NOM, VER:cond, NOM, ADJ, VER:p...",positive,0.071429,0.214286,0.071429,0.142857,0.142857,0.071429


In [298]:
final_df.to_csv('./submission.csv', index=False, encoding='utf8') # save csv

# Ontology

In [299]:
import owlready2
from owlready2 import *

In [300]:
onto = get_ontology("https://raw.githubusercontent.com/jannahastings/emotion-ontology/master/ontology/MFOEM.owl").load()

In [301]:
sentiments = ['joy', 'fear', 'sadness', 'anger', 'surprise', 'disgust']
for sentiment in sentiments:
  print(sentiment+' : '+ str(onto.search(label = sentiment)))

joy : [obo.MFOEM_000034]
fear : [obo.MFOEM_000026]
sadness : [obo.MFOEM_000056]
anger : [obo.MFOEM_000009]
surprise : [obo.MFOEM_000032]
disgust : [obo.MFOEM_000019]


In [302]:
tweets['max_emotion'] = tweets[['joy','fear','sadness','angry','surprise','disgust']].idxmax(axis=1)
tweets['IRI'] = tweets['max_emotion']

tweets['IRI'] = tweets['IRI'].replace(to_replace ="joy", value ="obo.MFOEM_000034")
tweets['IRI'] = tweets['IRI'].replace(to_replace ="fear", value ="obo.MFOEM_000026")
tweets['IRI'] = tweets['IRI'].replace(to_replace ="sadness", value ="obo.MFOEM_000056")
tweets['IRI'] = tweets['IRI'].replace(to_replace ="angry", value ="obo.MFOEM_000009")
tweets['IRI'] = tweets['IRI'].replace(to_replace ="surprise", value ="obo.MFOEM_000032")
tweets['IRI'] = tweets['IRI'].replace(to_replace ="disgust", value ="obo.MFOEM_000019")

tweets.tail(5)

,original,lemma,tag,polarity,joy,fear,sadness,angry,surprise,disgust,max_emotion,IRI
195,"[clemence_guette, raveaudgilles, richesflores,...","[clemence_guette, raveaudgilles, richesflores,...","[NOM, NOM, ADJ, NOM, NOM, NOM, NOM, ADJ, NOM, ...",positive,0.000000,0.111111,0.000000,0.111111,0.111111,0.111111,fear,obo.MFOEM_000026
196,"[clemence_guette, raveaudgilles, richesflores,...","[clemence_guette, raveaudgilles, richesflores,...","[NOM, NOM, ADJ, VER:pper, NOM, VER:pres, VER:i...",positive,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,surprise,obo.MFOEM_000032
197,"[clemence_guette, raveaudgilles, faut, revenir...","[clemence_guette, raveaudgilles, falloir, reve...","[NOM, NOM, VER:pres, VER:infi, NOM, VER:pres, ...",positive,0.076923,0.000000,0.076923,0.000000,0.000000,0.076923,joy,obo.MFOEM_000034
198,"[clemence_guette, raveaudgilles, richesse, pay...","[clemence_guette, raveaudgilles, richesse, pay...","[NOM, NOM, NOM, NOM, VER:cond, NOM, ADJ, VER:p...",positive,0.071429,0.214286,0.071429,0.142857,0.142857,0.071429,fear,obo.MFOEM_000026
199,"[clemence_guette, raveaudgilles, questions, éc...","[clemence_guette, raveaudgilles, question, éco...","[NOM, NOM, NOM, ADJ, VER:pres, ADV, VER:pper, ...",positive,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,sadness,obo.MFOEM_000056


# Done!